In [2]:

import open3d as o3d
import copy
import numpy as np
import surface_acquisition as surface
import visualization as vis

In [9]:
folder = "Pacients"
number = 66
patient = "BR0" + f"{number}"

digital_twin = f"{folder}/{patient}/Segment_4.stl"
digital_twin_1 = f"{folder}/{patient}/try.stl"
surface_scan_path = "Pacients/BR074/SS74/Scan 1.obj"
surface_digital_twin = f"{folder}/{patient}/Surface.ply"

In [10]:
pcd = surface.sample_mesh(digital_twin, 2000000)
rotated_pcd = surface.rotation_pcd(pcd)
diameter = surface.computation_diameter(rotated_pcd)
x_max, y_max, z_max, x_min, y_min, z_min = surface.bouding_points(rotated_pcd)

::       Reading mesh
::       Sampling Mesh
::       Rotation of the pcd
::      z_min and half_x: (-24.918293055954663, -36.56111027088511)
::      z_min and half_x: (0.0, 0.0)
::       Computing Diameter
::      Diameter of the pcd is: 591.6888632646235
::       Computing limits
::      Bounding Limits Max:[(230.14272482848574, 90.82725925377787, 260.5720210522331)
::      Bounding Limits Min:[(230.14272482848574, 174.3806546368459, 0.0)]


In [15]:
all_colors = [[1, 0.706, 0],  # Orange 
            [1, 0, 0],        # Red
            [1, 0, 0],        # Red
            [1, 0.5, 0],      # Orange
            [1, 0.5, 0],      # Orange
            [1, 1, 0],        # Yellow
            [1, 1, 0],        # Yellow
            [0, 1, 0],        # Green
            [0, 1, 0],        # Green
            [0, 0, 1],        # Blue
            [0, 0, 1],        # Blue
            [0.5, 0, 0.5],    # Indigo
            [0.5, 0, 0.5],    # Indigo
            [1, 0, 1],        # Violet
            [1, 0, 1],        # Violet
            [0.8, 0.6, 0.1],  # Gold
            [0.8, 0.6, 0.1],  # Gold
            [1, 0.706, 0],    # Orange 
            [1, 0.706, 0]    # Orange 
            ]   

In [16]:
all_cameras = {
    "camera_1": [0, 0, diameter/2],
    "camera_2": [(-diameter/2), y_max-y_min +abs(y_max-y_min)/4, diameter/5], 
    "camera_3": [(diameter/2), y_max-y_min + abs(y_max-y_min)/4 , diameter/5], 
    "camera_4": [(diameter/2), -100, (diameter - 120) * np.cos(30)],  
    "camera_5": [-(diameter/2), -100, (diameter - 120) * np.cos(30)],  
    "camera_6": [(diameter - 320)* np.cos(35), -y_min + abs(y_max-y_min)/6, (diameter - 320)],  
    "camera_7": [-(diameter - 320)* np.cos(35), -y_min + abs(y_max-y_min)/6, (diameter - 320)],  
    "camera_8": [(diameter/3) + x_max/3, -y_min + abs(y_max-y_min)/6, (diameter/3) - 50],  
    "camera_9": [-(diameter/3)-x_min/3, -y_min + abs(y_max-y_min)/6, (diameter/3) - 50],  
    "camera_10": [(diameter/3), 0, (3*diameter/7)],  
    "camera_11": [-(diameter/3), 0, (3*diameter/7)],  
    "camera_12": [(0.22*diameter), -y_min + abs(y_max-y_min)/6, 0.5*diameter],  
    "camera_13": [-(0.22*diameter), -y_min + abs(y_max-y_min)/6, (0.5 *diameter)], 
    "camera_14": [(diameter/3), -y_min + abs(y_max-y_min)/6, (diameter/2)],  
    "camera_15": [-(diameter/3), -y_min + abs(y_max-y_min)/6, (diameter/2)],  
    "camera_16": [(2*diameter/5), -y_min + abs(y_max-y_min)/6, (diameter/4)],  
    "camera_17": [-(2*diameter/5), -y_min + abs(y_max-y_min)/6, (diameter/4)],
    "camera_18": [0, -y_min + 60, 1.5 * z_max]
}

In [17]:
surface.visualize_hpr(rotated_pcd, all_cameras, all_colors)

::       Computing Diameter
::      Diameter of the pcd is: 591.6888632646235
::       Computation of the box
::       Computing Diameter
::      Diameter of the pcd is: 591.6888632646235
::      Point Cloud Number 1-th Done
::      Point Cloud Number 2-th Done
::      Point Cloud Number 3-th Done
::      Point Cloud Number 4-th Done
::      Point Cloud Number 5-th Done
::      Point Cloud Number 6-th Done
::      Point Cloud Number 7-th Done
::      Point Cloud Number 8-th Done
::      Point Cloud Number 9-th Done
::      Point Cloud Number 10-th Done
::      Point Cloud Number 11-th Done
::      Point Cloud Number 12-th Done
::      Point Cloud Number 13-th Done
::      Point Cloud Number 14-th Done
::      Point Cloud Number 15-th Done
::      Point Cloud Number 16-th Done
::      Point Cloud Number 17-th Done
::      Point Cloud Number 18-th Done


In [ ]:
surface.final_pcd(rotated_pcd, all_cameras, all_colors, patient)

In [ ]:

vis.visualize_surface_and_all(surface_digital_twin,digital_twin)

In [12]:
# Given a x = k and bounded yy ans zz makes a plane
def make_plane(plane, y_max, y_min, z_max, z_min):
    y_range = np.linspace(y_min, y_max, 500)
    z_range = np.linspace(z_min, z_max, 500)

    plane_points = [[plane, y, z] for y in y_range for z in z_range]
    plane_cloud = o3d.geometry.PointCloud()
    plane_cloud.points = o3d.utility.Vector3dVector(plane_points)
    plane_cloud.paint_uniform_color((0,0,0))
    return plane_cloud

In [13]:
def visualize_plane_point_cloud(pcd, planes):
    _, y_max, z_max, _, y_min, z_min = surface.bouding_points(pcd)
    various_planes = {}
    for i, plane in enumerate(planes):
        various_planes[f'plane_{i}'] = make_plane(plane, y_max, -y_min, z_max, -z_min)
    
    # Visualize all planes and the point cloud
    all_geometries = [pcd] + list(various_planes.values())
    o3d.visualization.draw_geometries(all_geometries)

In [14]:
visualize_plane_point_cloud(rotated_pcd, [-150,-50,0,50,150])

::       Computing limits
::      Bounding Limits Max:[(230.14272482848574, 90.82725925377787, 260.5720210522331)
::      Bounding Limits Min:[(230.14272482848574, 174.3806546368459, 0.0)]


In [ ]:
def planes(planes_list):
    _, y_max, z_max, _, y_min, z_min = surface.bouding_points(pcd)
    various_planes = {}
    for i, plane in enumerate(planes_list):
        various_planes[f'plane_{i}'] = make_plane(plane, y_max, -y_min, z_max, -z_min)
    
    all_geometries = list(various_planes.values())
    return all_geometries

In [15]:

        
# visualize_plane_point_cloud(rotated_pcd, [-150,0,150])
all_planes = planes([0,50])
intersections_kdtree(rotated_pcd, all_planes, 0.02)


::       Computing limits
::      Bounding Limits Max:[(230.19579525036687, 90.80237997823457, 260.5767409635464)
::      Bounding Limits Min:[(230.19579525036687, 174.3847327629589, 0.0)]


TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. open3d.cpu.pybind.geometry.KDTreeFlann()
    2. open3d.cpu.pybind.geometry.KDTreeFlann(data: numpy.ndarray[numpy.float64[m, n]])
    3. open3d.cpu.pybind.geometry.KDTreeFlann(geometry: open3d.cpu.pybind.geometry.Geometry)
    4. open3d.cpu.pybind.geometry.KDTreeFlann(feature: open3d::pipelines::registration::Feature)

Invoked with: [PointCloud with 1000000 points., PointCloud with 1000000 points.]

: 

In [19]:
def min_y(pcd, plane):
    plane_cloud = o3d.geometry.PointCloud()
    plane_cloud.points = o3d.utility.Vector3dVector([[plane, y, z] for y in y_range for z in z_range])
    distances = np.asarray(pcd.compute_point_cloud_distance(plane_cloud))
    min_distance_index = np.argmin(distances)
    min_point = pcd.points[min_distance_index]
    min_y = min_point[1]
    print(f"Minimum y-value for plane {plane}:", min_y)
    return min_y

In [21]:
min_y(rotated_pcd, -150)
min_y(rotated_pcd, -100)
min_y(rotated_pcd, -50)
min_y(rotated_pcd, 0)
min_y(rotated_pcd, 50)
min_y(rotated_pcd, 100)
min_y(rotated_pcd, 150)

Minimum y-value for plane 1: -22.84284923474232
Minimum y-value for plane 1: 11.143860948968886
Minimum y-value for plane 1: 43.52759787225349
Minimum y-value for plane 1: -22.572363815355644
Minimum y-value for plane 1: -134.32227934988572
Minimum y-value for plane 1: 8.744476354406899
Minimum y-value for plane 1: -55.19740475477418


-55.19740475477418

Dataset Uploading

In [18]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import features
import filter

folder = "Pacients"
number = 74
patient = "BR0" + f"{number}"
surface_digital_twin_path = f"{folder}/{patient}/Surface.ply"
pcd = o3d.io.read_point_cloud(surface_digital_twin_path)

Downsample

In [19]:
voxel_down_pcd = pcd.voxel_down_sample(voxel_size=0.5)
# o3d.visualization.draw_geometries([voxel_down_pcd])

In [ ]:

uni_down_pcd = pcd.uniform_down_sample(every_k_points=20)
# o3d.visualization.draw_geometries([uni_down_pcd])

Outlier Removal

In [20]:
cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=1.0)
filter.display_inlier_outlier(voxel_down_pcd, ind)

Showing outliers (red) and inliers (gray): 


PointCloud with 401626 points.

In [ ]:
cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=5, radius=5)
filter.display_inlier_outlier(voxel_down_pcd, ind)

In [21]:
pcd_median = filter.median_filter(voxel_down_pcd, 7)

In [26]:
cl, ind = pcd_median.remove_statistical_outlier(nb_neighbors=100,
                                                        std_ratio=0.5)
inlier_cloud = filter.display_inlier_outlier(pcd_median, ind)
print(inlier_cloud)

Showing outliers (red) and inliers (gray): 
PointCloud with 375848 points.


2. Features Display

In [25]:
fpfh_digital_twin = features.fpfh(inlier_cloud, 0.5)
features.save_feature_image(inlier_cloud, fpfh_digital_twin, 0.5, "Features", patient)

Computing Fast Points Feature Histogram...
FPFH Done
Feature number 0-th
Feature number 1-th
Feature number 2-th
Feature number 3-th
Feature number 4-th
Feature number 5-th
Feature number 6-th
